In [ ]:
%%capture
!pip install mne #The MNE Package is installed
FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

#Import

In [ ]:
from google.colab.drive import mount
import numpy as np
import os
from time import time
from MI_EEG_ClassMeth.FeatExtraction import Power_based_Connectivities, Phase_based_Connectivities, MotifSynchronization
from MI_EEG_ClassMeth.Preprocessing import Flatten
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.metrics import cohen_kappa_score, roc_auc_score, make_scorer
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from MI_EEG_ClassMeth.utils import grid_search_info
from pickle import dump as pdump
from joblib import dump as jdump

#Mount Drive

In [ ]:
mount("/content/drive", force_remount=True)

Mounted at /content/drive


#Power Based Connectivities

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pearson/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pearson/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pearson/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pearson/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/spearman/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/spearman/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/spearman/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/spearman/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/gfc/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/gfc/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/gfc/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/gfc/ica_sl/')

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/pearson/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/spearman/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/gfc/')

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/'

seed = 23

subjects = np.arange(52) + 1
subjects = np.delete(subjects,[28,33])

pbc = Power_based_Connectivities()
flatt = Flatten()

clf = LDA()

hyparams = {}

k = 10
test_size=0.2
cv = StratifiedShuffleSplit(n_splits=k, test_size=test_size, random_state=seed)

scores ={'acc': 'accuracy', 'kappa':  make_scorer(cohen_kappa_score), 'auc': make_scorer(roc_auc_score, average='macro', multi_class='ovo')}   

grid_search = GridSearchCV(clf, hyparams, cv=cv, verbose=0, scoring=scores,
                          refit='acc', error_score='raise', n_jobs=-1, return_train_score=True)

# ------------------------------------------------------------------------------
t_i = time()
for sbj in subjects[30:]:
  y_dir = os.path.join(data_dir , 'labels/')
  y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

  for type_con in ['pearson', 'spearman', 'gfc']:
    cv_results = {'params':[], 'mean_train_acc':[], 'std_train_acc':[], 'mean_train_kappa':[], 'std_train_kappa':[], 'mean_train_auc':[], 'std_train_auc':[],
                  'mean_test_acc':[], 'std_test_acc':[], 'mean_test_kappa':[], 'std_test_kappa':[], 'mean_test_auc':[], 'std_test_auc':[]}

    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/'  + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = pbc.transform(X, type_con=type_con, gammad=1)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)

        grid_search.fit(flat_C_xx_w_f, y)
        best_params, (mean_test, std_test), (mean_train, std_train), _, _  = grid_search_info(grid_search.cv_results_, ['acc', 'kappa', 'auc'])
        cv_results['params'].append((ica, sl) + tuple(best_params.values()))
        cv_results['mean_train_acc'].append(round(mean_train[0]*100, 1))
        cv_results['std_train_acc'].append(round(std_train[0]*100, 1))
        cv_results['mean_train_kappa'].append(round(mean_train[1], 2))
        cv_results['std_train_kappa'].append(round(std_train[1], 2))
        cv_results['mean_train_auc'].append(round(mean_train[2], 2))
        cv_results['std_train_auc'].append(round(std_train[2], 2))
        cv_results['mean_test_acc'].append(round(mean_test[0]*100, 1))
        cv_results['std_test_acc'].append(round(std_test[0]*100, 1))
        cv_results['mean_test_kappa'].append(round(mean_test[1], 2))
        cv_results['std_test_kappa'].append(round(std_test[1], 2))
        cv_results['mean_test_auc'].append(round(mean_test[2], 2))
        cv_results['std_test_auc'].append(round(std_test[2], 2))

        jdump(grid_search.best_estimator_, models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
    
    with open(results_dir + type_con + '/' + 'sbj' + str(sbj) + '.txt', 'wb') as f:
      pdump(cv_results, f)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 50.82 m


#Phase Based Connectivities

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/coh/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/coh/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/coh/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/coh/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/plv/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/plv/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/plv/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/plv/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pli/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pli/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pli/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/pli/ica_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/wpli/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/wpli/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/wpli/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/wpli/ica_sl/')

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/coh/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/plv/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/pli/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/wpli/')

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/'

seed = 23

subjects = np.arange(52) + 1
subjects = np.delete(subjects,[28,33])

pbc = Phase_based_Connectivities()
flatt = Flatten()

clf = LDA()

hyparams = {}

k = 10
test_size=0.2
cv = StratifiedShuffleSplit(n_splits=k, test_size=test_size, random_state=seed)

scores ={'acc': 'accuracy', 'kappa':  make_scorer(cohen_kappa_score), 'auc': make_scorer(roc_auc_score, average='macro', multi_class='ovo')} 

grid_search = GridSearchCV(clf, hyparams, cv=cv, verbose=0, scoring=scores,
                          refit='acc', error_score='raise', n_jobs=-1, return_train_score=True)

# ------------------------------------------------------------------------------
t_i = time()
for sbj in subjects[32:40]:
  y_dir = os.path.join(data_dir , 'labels/')
  y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

  for type_con in ['coh', 'plv', 'pli', 'wpli']:
    cv_results = {'params':[], 'mean_train_acc':[], 'std_train_acc':[], 'mean_train_kappa':[], 'std_train_kappa':[], 'mean_train_auc':[], 'std_train_auc':[],
                  'mean_test_acc':[], 'std_test_acc':[], 'mean_test_kappa':[], 'std_test_kappa':[], 'mean_test_auc':[], 'std_test_auc':[]}

    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/'  + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = pbc.transform(X, type_con=type_con)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)

        grid_search.fit(flat_C_xx_w_f, y)
        best_params, (mean_test, std_test), (mean_train, std_train), _, _  = grid_search_info(grid_search.cv_results_, ['acc', 'kappa', 'auc'])
        cv_results['params'].append((ica, sl) + tuple(best_params.values()))
        cv_results['mean_train_acc'].append(round(mean_train[0]*100, 1))
        cv_results['std_train_acc'].append(round(std_train[0]*100, 1))
        cv_results['mean_train_kappa'].append(round(mean_train[1], 2))
        cv_results['std_train_kappa'].append(round(std_train[1], 2))
        cv_results['mean_train_auc'].append(round(mean_train[2], 2))
        cv_results['std_train_auc'].append(round(std_train[2], 2))
        cv_results['mean_test_acc'].append(round(mean_test[0]*100, 1))
        cv_results['std_test_acc'].append(round(std_test[0]*100, 1))
        cv_results['mean_test_kappa'].append(round(mean_test[1], 2))
        cv_results['std_test_kappa'].append(round(std_test[1], 2))
        cv_results['mean_test_auc'].append(round(mean_test[2], 2))
        cv_results['std_test_auc'].append(round(std_test[2], 2))

        jdump(grid_search.best_estimator_, models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
    
    with open(results_dir + type_con + '/' + 'sbj' + str(sbj) + '.txt', 'wb') as f:
      pdump(cv_results, f)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 193.97 m


#Information Based Connectivities

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/motif/none_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/motif/ica_none/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/motif/none_sl/')
os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/motif/ica_sl/')

os.makedirs('/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/motif/')

In [ ]:
data_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/data/'
models_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/models/'
results_dir = '/content/drive/Shareddrives/GCPDS/users/Mateo/GIGA_MI_ME_preprocessing/results/hyparam_tuning/'

seed = 23

subjects = np.arange(52) + 1
subjects = np.delete(subjects,[28,33])

mt = MotifSynchronization()
flatt = Flatten()

clf = LDA()

hyparams = {}

k = 10
test_size=0.2
cv = StratifiedShuffleSplit(n_splits=k, test_size=test_size, random_state=seed)

scores ={'acc': 'accuracy', 'kappa':  make_scorer(cohen_kappa_score), 'auc': make_scorer(roc_auc_score, average='macro', multi_class='ovo')} 

grid_search = GridSearchCV(clf, hyparams, cv=cv, verbose=0, scoring=scores,
                          refit='acc', error_score='raise', n_jobs=-1, return_train_score=True)

# ------------------------------------------------------------------------------
t_i = time()
for sbj in subjects:
  y_dir = os.path.join(data_dir , 'labels/')
  y = np.load(y_dir + 'sbj' + str(sbj) + '.npy')

  for type_con in ['motif']:
    cv_results = {'params':[], 'mean_train_acc':[], 'std_train_acc':[], 'mean_train_kappa':[], 'std_train_kappa':[], 'mean_train_auc':[], 'std_train_auc':[],
                  'mean_test_acc':[], 'std_test_acc':[], 'mean_test_kappa':[], 'std_test_kappa':[], 'mean_test_auc':[], 'std_test_auc':[]}

    for ica in ['none_', 'ica_']:
      for sl in ['none', 'sl']:       
        X = np.load(data_dir + ica+sl + '/'  + 'sbj' + str(sbj) + '.npy')
        C_xx_w_f = mt.transform(X)
        flat_C_xx_w_f = flatt.transform(C_xx_w_f)

        grid_search.fit(flat_C_xx_w_f, y)
        best_params, (mean_test, std_test), (mean_train, std_train), _, _  = grid_search_info(grid_search.cv_results_, ['acc', 'kappa', 'auc'])
        cv_results['params'].append((ica, sl) + tuple(best_params.values()))
        cv_results['mean_train_acc'].append(round(mean_train[0]*100, 1))
        cv_results['std_train_acc'].append(round(std_train[0]*100, 1))
        cv_results['mean_train_kappa'].append(round(mean_train[1], 2))
        cv_results['std_train_kappa'].append(round(std_train[1], 2))
        cv_results['mean_train_auc'].append(round(mean_train[2], 2))
        cv_results['std_train_auc'].append(round(std_train[2], 2))
        cv_results['mean_test_acc'].append(round(mean_test[0]*100, 1))
        cv_results['std_test_acc'].append(round(std_test[0]*100, 1))
        cv_results['mean_test_kappa'].append(round(mean_test[1], 2))
        cv_results['std_test_kappa'].append(round(std_test[1], 2))
        cv_results['mean_test_auc'].append(round(mean_test[2], 2))
        cv_results['std_test_auc'].append(round(std_test[2], 2))

        jdump(grid_search.best_estimator_, models_dir + type_con + '/' + ica+sl + '/'+ 'sbj' + str(sbj) + '.joblib')
    
    with open(results_dir + type_con + '/' + 'sbj' + str(sbj) + '.txt', 'wb') as f:
      pdump(cv_results, f)

t_f = time() - t_i
print('time:',round(t_f/60, 2),'m')

time: 91.94 m
